# Esercitazione 1.1/1.2: Valori di similarità e spiegazione

Per questo esercizio andremo a importare una serie di significati raccolti a lezione e calcolare una similarità tra i termini basandoci su tali definizioni.

Tenteremo tre approcci per calcolare la similarità: i primi due prevedono il conteggio delle parole comuni tra le varie definizioni rispetto al numero totale, il primo approccio come vedremo non darà i risultati sperati per via della diversità di definizioni create. Evolveremo quell'idea nel secondo approccio dove useremo un parametro "percentuale" per aumentare il numero di parole accettate nell'insieme di termini comuni, quindi anche parole che non compaiono in tutte le definizioni.

In fine per il terzo approccio saranno utilizzate le librerie di `sklearn` per creare vettori e calcolarne la *cosine similarity*

Iniziamo a creare la nostra *dictionary* per importare le definizioni dal file `defs.csv`:

In [1]:
import csv
import os
from pprint import pprint

meanings_map = {}
with open(os.getcwd()+'\\defs.csv', "r", encoding="utf8") as csv_file:
    words = csv_file.readline().replace('\n','').split(',')
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        for i in range(1, 5):
            if not meanings_map.keys().__contains__(words[i]):
                meanings_map[words[i]] = []
            meanings_map[words[i]].append(row[i]) if len(row[i]) > 0 else None

# Example
pprint (meanings_map, width=1000)

{'Apprehension': ['something strange, which causes a strange feeling of strangeness, very different from normal, abnormal',
                  'fearful expectation or anticipation',
                  'A moode where one feel agitation',
                  'State of disturbance',
                  'Worry about the future',
                  'act of understanding something, or the way that something is understood',
                  'Non-relaxed state of mind derived from unaccommodating events',
                  'Sense of loss, sadness, fear and awe',
                  'Mental status that make a person feel uncofortable about something or a situation',
                  'state of mind in which one is frightened',
                  'anxiety or fear that something bad or unexpected will happen.',
                  'Mental state of high anxiety',
                  'feeling of preoccupation about something or someone',
                  'is the emotional state of a person who feels fear',
   

## Preprocessing

Nella seguente fase andiamo a ricostruire la dictionary precedentemente estratta dal file input andando ad attuare le seguenti modifiche:

    - Rimozione delle stopword, estratte dal corpus nltk
    - Tokenizzation per ogni definizione
    - Filtriamo le parole che non sono alfanumeriche
    - Memorizziamo lo *stemma* dei termini finali

In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

ps = PorterStemmer()
stop_words = set(stopwords.words('english'))
for key in meanings_map:
    for definition in meanings_map.get(key):
        word_tokens = word_tokenize(definition)
        filtered_sentence = [ps.stem(w.lower()) for w in word_tokens if not w.lower() in stop_words and w.lower().isalnum()]
        meanings_map.get(key)[meanings_map.get(key).index(definition)] = filtered_sentence

# Example
pprint (meanings_map, width=1000)

{'Apprehension': [['someth', 'strang', 'caus', 'strang', 'feel', 'strang', 'differ', 'normal', 'abnorm'],
                  ['fear', 'expect', 'anticip'],
                  ['mood', 'one', 'feel', 'agit'],
                  ['state', 'disturb'],
                  ['worri', 'futur'],
                  ['act', 'understand', 'someth', 'way', 'someth', 'understood'],
                  ['state', 'mind', 'deriv', 'unaccommod', 'event'],
                  ['sens', 'loss', 'sad', 'fear', 'awe'],
                  ['mental', 'statu', 'make', 'person', 'feel', 'uncofort', 'someth', 'situat'],
                  ['state', 'mind', 'one', 'frighten'],
                  ['anxieti', 'fear', 'someth', 'bad', 'unexpect', 'happen'],
                  ['mental', 'state', 'high', 'anxieti'],
                  ['feel', 'preoccup', 'someth', 'someon'],
                  ['emot', 'state', 'person', 'feel', 'fear'],
                  ['feel', 'someth', 'bad', 'happen'],
                  ['neg', 'emot', 'perso

Costruiamo un metodo di stampa per semplificare la lettura dei risultati e mostrare una possibile spiegazione dei dati ottenuti:

In [3]:
def print_result(word, intersection_list, similarity, percentage=False):
    print(f"Result for term: {word}")
    print(f"Percentage used for filtering: {percentage}") if percentage else None
    print(f"Intersection list: {intersection_list}")
    print(f"Similarity: {similarity}")
    print("\n\n")


## Primo tentativo:

Il primo metodo provato per calcolare la similarità consiste nel creare una lista intersezione con tutte le parole presenti in ogni definizione e dividere quel valore per il numero di parole medio per ogni definizione:
$$
Sim = \frac{Intersezione}{Media} \\
$$

Intersezione = parole comuni a tutte le definizioni <br>
Media = media delle lunghezze delle definizioni

    

In [4]:
def calculate_normal_similarity(meanings_map):
    for word in meanings_map:
        definition_lists = meanings_map.get(word)
        sum_len = 0
        intersection_list = definition_lists[0]
        for definition in definition_lists:
            intersection_list = [word for word in intersection_list if word in set(definition)]
            sum_len += definition.__len__()
        similarity = len(intersection_list) / ( sum_len / len(definition_lists))
        print_result(word, intersection_list, similarity)
        
# Example
calculate_normal_similarity(meanings_map)

Result for term: Courage
Intersection list: []
Similarity: 0.0



Result for term: Paper
Intersection list: []
Similarity: 0.0



Result for term: Apprehension
Intersection list: []
Similarity: 0.0



Result for term: Sharpener
Intersection list: []
Similarity: 0.0





Come prevedibile questo approccio non porta risultati perchè è molto difficile che delle persone portino a scrivere descrizioni di termini così simili e usando le stesse parole.

## Secondo tentativo:

E' necessario allargare la restrinzione sulla lista di intersezione, in altre parole dobbiamo accettare anche termini che non compaiono in tutte le definizione ma comunque che siano molto diffusi.

Introduciamo un valore di percentuale, accettiamo tutte le parole che son presenti per esempio nel 50% delle frasi.

Il valore *0.5* è stato ottenuto su diversi tentativi ed è stato scelto perchè è quello che sembra dare risultati più rappresentativi

$$
Sim = \frac{Intersezione}{Media} \\
$$

Intersezione = parole in almeno 50% delle definizioni <br>
Media = media delle lunghezze delle definizioni

In [5]:
from collections import Counter

def calculate_percentage_similarity(meanings_map, percentage):
    for word in meanings_map:
        definition_lists = meanings_map.get(word)
        word_counter = Counter(x for xs in definition_lists for x in set(xs))
        intersection_list = [word for word in word_counter.keys() if word_counter.get(word)/len(definition_lists) > percentage]
        sum_len = sum(len(x) for x in definition_lists)
        similarity = len(intersection_list) / (sum_len / len(definition_lists))
        print_result(word, intersection_list, similarity, percentage)
        
# Example
calculate_percentage_similarity(meanings_map, 0.5)

Result for term: Courage
Percentage used for filtering: 0.5
Intersection list: ['fear', 'abil']
Similarity: 0.3846153846153846



Result for term: Paper
Percentage used for filtering: 0.5
Intersection list: ['materi', 'write']
Similarity: 0.4137931034482759



Result for term: Apprehension
Percentage used for filtering: 0.5
Intersection list: []
Similarity: 0.0



Result for term: Sharpener
Percentage used for filtering: 0.5
Intersection list: ['pencil', 'tool', 'sharpen', 'use']
Similarity: 0.8823529411764706





Possiamo notare dalla stampa su *Intersection list* che compaiono i termini più usati nelle definizioni e variando il valore della percentuale possiamo notare quali son più usati o meno

Questa informazione ci permette di capire come termini più concreti o comunque di cui si ha una chiara immagine del loro aspetto e/o uso tendano ad avere una lista più lunga, come nel caso di *Sharpener*.<br>
Al contrario *Apprehension* anche scendendo ad una percentuale di 0.4, quindi accettando parole che compaiono nel 40% delle definizioni non riesce a trovare delle parole comuni.<br>
Tuttavia il rapporto tra i vari termini resta sempre lo stesso:
    Le definizioni associate alle parole più concrete e specifiche tendono ad ottenere una similarità maggiore

In [6]:
# Example
calculate_percentage_similarity(meanings_map, 0.4)

Result for term: Courage
Percentage used for filtering: 0.4
Intersection list: ['fear', 'abil']
Similarity: 0.3846153846153846



Result for term: Paper
Percentage used for filtering: 0.4
Intersection list: ['materi', 'use', 'write']
Similarity: 0.6206896551724138



Result for term: Apprehension
Percentage used for filtering: 0.4
Intersection list: []
Similarity: 0.0



Result for term: Sharpener
Percentage used for filtering: 0.4
Intersection list: ['pencil', 'tool', 'sharpen', 'use']
Similarity: 0.8823529411764706





Diminuendo ulteriormente la percentuale riusciamo ad ottenere dei termini comuni per *Apprehension*. Tuttavia una percentuale troppo bassa tende a restituire risultati meno affidabili soggetti all'uso comune di certi termini nella lingua italiana.
Notiamo comunque che il rapporto resta costante:

In [7]:
# Example
calculate_percentage_similarity(meanings_map, 0.35)

Result for term: Courage
Percentage used for filtering: 0.35
Intersection list: ['fear', 'abil']
Similarity: 0.3846153846153846



Result for term: Paper
Percentage used for filtering: 0.35
Intersection list: ['materi', 'use', 'write']
Similarity: 0.6206896551724138



Result for term: Apprehension
Percentage used for filtering: 0.35
Intersection list: ['fear']
Similarity: 0.22556390977443608



Result for term: Sharpener
Percentage used for filtering: 0.35
Intersection list: ['pencil', 'tool', 'sharpen', 'object', 'use']
Similarity: 1.1029411764705883





## Terzo tentativo

Come ultimo tentativo si sfrutteranno le librerie di `sklearn` per costruire dei vettori delle definizioni da confrontare tramite similarità del coseno.

Questa soluzione è stata presentata per mostrare la coerenza con i dati ottenuti dai metodi precedenti, ovvero il rapporto tra i quattro termini presentati.

Il primo metodo consiste nel creare un vettore di occorrenze delle parole, è stato necessario catturare dei Warning legati alla versione utilizzata per non complicare la lettura dei risultati.

Dal momento che il risultato della libreria è una matrice diagonale di similarità tra termini si è costruito un metodo `get_mean_diagonal_matrix` per calcolarne la media.

Tuttavia questo approccio non ci permette di dare una spiegazione dei risultati mostrando i termini comuni ma è una buona metrica di confronto per i risultati ottenuti precedentemente.

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy
import warnings

def get_vectors(text):
    # Son presenti dei warning di tipo FutureWarning:
    # From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
    # per agevolare la lettura li filtriamo
    # Questa fase prima crea un elenco di termini in tutto il doc per settare la lunghezza della matrice e poi 
    # per ogni frase inserisce nella posizione il count di quante volte il termine è trovato
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        vectorizer = CountVectorizer()
    return vectorizer.fit_transform(text).toarray()

def get_mean_diagonal_matrix(matrix):
    list_term = []
    for i in range(0, len(matrix)):
        list_term = numpy.append(list_term, matrix[i][i+1:len(matrix)])
    return sum(list_term) / len(list_term)


def get_cosine_sim(meanings_map):
    for word in meanings_map:
        input_list = [' '.join(definition) for definition in meanings_map.get(word)]
        vectors = get_vectors(input_list)
        res = cosine_similarity(vectors)
        print_result(word, None, get_mean_diagonal_matrix(res))
        
# Example
get_cosine_sim(meanings_map)

Result for term: Courage
Intersection list: None
Similarity: 0.20125209518499848



Result for term: Paper
Intersection list: None
Similarity: 0.25739573551431894



Result for term: Apprehension
Intersection list: None
Similarity: 0.12027347231240926



Result for term: Sharpener
Intersection list: None
Similarity: 0.4203289919882313



